In [212]:
import requests
import networkx as nx
import pandas as pd
import math
from sklearn.metrics.pairwise import cosine_similarity

In [121]:
headers = {'x-api-key': '5UgDqL5l2j4y9Di50fuPk7o0GF3ybgN83FTeEsLC'}

In [21]:
citation_prediction = pd.read_json('test_qrel.jsonl', lines=True)

In [108]:
citation_meta = pd.read_json('meta.jsonl', lines=True)

In [22]:
def get_query_candidates(df):
    positive_candidates = {}
    all_candidates = {}
    for query, query_df in df.groupby('query_id'):
        positive_candidates[query] = query_df[query_df['score'] == 1]['cand_id'].tolist()
        all_candidates[query] = query_df['cand_id'].tolist()
    return positive_candidates, all_candidates

In [23]:
# Key: query_id Value: positive_candidates, all_candidates 
positive_candidates, all_candidates = get_query_candidates(citation_prediction)

In [124]:
# Download specter embeddings from semantic scholar
import time
specter_embeddings = {}
i = 0
for query_id, candidates in all_candidates.items():
    start = time.time()
    embeddings_list = []
    for candidate in candidates:
        url = 'https://api.semanticscholar.org/graph/v1/paper/{}?fields=embedding'.format(candidate)
        response = requests.get(url, headers=headers)
        if response.text == '{"message":"Too Many Requests"}':
            print("Too Many Requests.")
        embeddings_list.append(response.text)
    specter_embeddings[query_id] = embeddings_list
    end = time.time()
    print(i, ' ', query_id, ' done in ', end - start)
    i += 1

0   01273bd34dacfe9ef887b320f36934d2f9fa9b34  done in  6.910153865814209
1   012e396b02aa584cb74a65ae14af355e7c897858  done in  6.659092903137207
2   01d208b33561362f7714f714d3bc4a1f7aa1637c  done in  6.794214963912964
3   0210b3fe6f7173c86936b5dd9261bc0be0c45652  done in  7.118221044540405
4   0229829e9a1eed5769a2b5eccddcaa7cd9460b92  done in  6.778856992721558
5   027e7780dbda48d99f3654e77b4a63063224950e  done in  6.569776773452759
6   02c9fe33a5d8cb94373cea20a53f01e0a0e70f7f  done in  7.3612658977508545
7   030ff7012b92b805a60976f8dbd6a08c1cecebe6  done in  6.675525903701782
8   031a0f18d46b8e006eb4262233f7734fe4505c21  done in  6.658522844314575
9   033897d56c7cf4f084dec1fad072f1a6aca65c6e  done in  6.980877876281738
10   03725753e46ee9b13cbdfa78c9b62700d4cc2956  done in  6.561847925186157
11   038ee3d9e0a739752f4a270548ab8c97ed024633  done in  6.59473991394043
12   03bd09f62445ee68095f20000342c1c76b57d7c9  done in  6.7078680992126465
13   03d23160e7066e5adab0d55779287e3c4982b9d5  

111   19d9d5e14d4d3f3e4ef66e00cdbc11ff6a93e9d5  done in  6.75650691986084
112   1a278dcdc031dba7d9aa055aa3b450339ddbe161  done in  6.846771001815796
113   1a66df57f88e689438d505474eff73e3a9180c5b  done in  6.236313819885254
114   1b1aed7e508dce5e1c2d847ee69b8979e3be69d2  done in  6.5627031326293945
115   1b44df3232099b13e3aac7fc5811a946133d6db6  done in  6.973069906234741
116   1b4ed4a45a900d6a02f929f873cb25f51a0e054b  done in  6.845956802368164
117   1b7db8ad49f94da9b90db89bede5f27644bb9911  done in  6.549993276596069
118   1b9ce6abc0f3024b88fcd4dbd0c10cf5bcf7d38d  done in  6.518396854400635
119   1ba7a9c0e658a0d98253f999bf43c2e98c07f4e0  done in  6.55689001083374
120   1bc5ea91875143e0526ac587b9c4aa6c64ceec98  done in  7.022217035293579
121   1bf06b2bbf53089180fd3676720d80cbaed5975d  done in  6.777933120727539
122   1c0e8c3fb143eb5eb5af3026eae7257255fcf814  done in  7.228182077407837
123   1c454ae4e1bbc600791f3a4796fdb6b1ee2ca016  done in  6.8577892780303955
124   1cb77e2a09db58e9e8b

221   3131776d3b838e3a22ed9ccd7b7b92a41410df28  done in  7.40273904800415
222   3136f52540fb7925998eff94e9c016d2e1a6fe12  done in  6.942647218704224
223   313b369196df6026a97016327f8e4eebaf1a0176  done in  7.37047004699707
224   314d65679c866483277fcc209ad89e7abab20126  done in  7.235403060913086
225   3159c9423862b22c6326801ad4353ae2cfe30d32  done in  6.79935884475708
226   318e36796798d173020e2c92ea4188b60d84a450  done in  6.579757928848267
227   31c1327754c921e848bf2d0ff6ea6828e29680e3  done in  7.461673021316528
228   31c939e469b8910eb49af18247d68981cff1887a  done in  7.47302508354187
229   31d33747d8fff0b7a0c40dcf9944015af9a15b1a  done in  7.300878047943115
230   3343804410013190447ce2dc0d8fcf792916aa21  done in  7.060659885406494
231   335ad0ff82c728aca99fb0059c607cb18129526f  done in  7.446602821350098
232   337f48a82d9d2739d35b41cfc4e8cd3dc906639d  done in  6.952491998672485
233   33812e0c6b8fdc5e414c5eae760e574e5a81190a  done in  6.653756856918335
234   3388d516fc26536423b03e1

331   4877d14b881afad4476891e16e44bf00040f68c0  done in  6.946261882781982
332   48a01a7d5e01ebec4311495b1e45c1b59151efae  done in  6.717087984085083
333   490ec36471275164d104f155ef7ebbc70c5f7eea  done in  6.638592004776001
334   498aad3cb4d2b8e8ff1e68a40be7dec27e8139f4  done in  6.451720952987671
335   49aa693db2d6d6a9486a9c9b6e1ac653fe12ab97  done in  6.6123292446136475
336   4a343313042b654c7d85313de3d88486a1d0e9de  done in  6.667707204818726
337   4a6955d29a72953bb8ad36240615c55c167a0d43  done in  7.089130163192749
338   4ab6e67519411dbdb69c6111a3b8d5f334ebb579  done in  6.576978921890259
339   4acb38f96f3f69a443e1684040c14d1cecc4f841  done in  6.9798407554626465
340   4b450424be82fccb46a97bdad24a44f547d604d9  done in  6.6250810623168945
341   4b5d2c4bdef08a20ee6bea555de64425f003e46b  done in  6.4809699058532715
342   4b80771429b786ea87740738378ba1eb1504a57f  done in  6.461540937423706
343   4b82b3bf2b30d74c083940a86db5b09e4b81f0af  done in  6.673651933670044
344   4b96b327c8c128e

441   645802c55d809d5bf27f391837078689f7bf2333  done in  6.532149076461792
442   648678f1edab0d2139958070744b826d2b24c79e  done in  30.921674966812134
443   64982c97190bbd8b70beeed572392d0f308a93c2  done in  38.113242864608765
444   64abfbc0b66ec8579a92a33dca19e92bc2095ea2  done in  6.202564239501953
445   64acf2715c5e8fc54e444ca38c39f41b89b6cb19  done in  6.237473964691162
446   64da1980714cfc130632c5b92b9d98c2f6763de6  done in  6.822070121765137
447   64ddfb947878347a30610b6ca2314fe2bac96a4a  done in  6.744611024856567
448   6519bf5580fcdcc9c50fd72c6c8dc5d040d443e8  done in  6.767537832260132
449   65a858ca95dcfa032e812a7f1fc7ee5bdac88f5b  done in  7.05499005317688
450   661a03482f0532b7355188059b2263091ade1bae  done in  7.301554918289185
451   663e06e02a306660ef508aaf2ffaf523c6e96088  done in  6.895367860794067
452   665d50db7006e2595c7d55687fed7329192bd71e  done in  7.161175966262817
453   66b64087bfd9a59805a24ec06add72dac6e86fc9  done in  6.5314881801605225
454   66ef0f611b2b5b22d

551   7d4c8427e3bdc307e8da6cf53110aafb78f47a8a  done in  6.66909384727478
552   7d6782b0ca41fd3ce99f3913d03c576d5fe65647  done in  6.611979961395264
553   7db20fcc1d71edc32c365f145148d24b9f1427a5  done in  6.97914981842041
554   7dcb308b9292a8bc87d6f7793d2ca5e0e19dfa40  done in  6.889673948287964
555   7ddc47d09d89a89c6a1c7643895a173aba07173f  done in  7.001248121261597
556   7de4eb8c11972a64236434cbe95af47b92d438b9  done in  7.271974802017212
557   7e3b5f0ffec62944f4b97a50553a12a39f54ba4a  done in  7.216969966888428
558   7e475e3b326d4fc9f2e908a3222796031d94aec0  done in  6.433187246322632
559   7e7f843b9638ee9dc321fcd348ea2185b9126854  done in  6.737378120422363
560   7faf5d0915d604821d7ad2ade77f81a751309eee  done in  6.518041133880615
561   7ff20fc82c95eda6645ff2570279251bb8435ec5  done in  6.507144927978516
562   7ffd6b7c27558400bfe609161b158d6a34b62f62  done in  6.946717262268066
563   80093393acfaf79161a969d5180d0fd561ee853b  done in  16.776176929473877
564   802f3f316f87c6bc675c

661   994ca78751ecfc571cb7ce05c4343c12b9677b71  done in  6.626866817474365
662   99838f4d4259f67f903f8f762841499513b0b511  done in  7.062068223953247
663   9992dfb672e93683f21c571624a4e13d681d79b7  done in  7.040365934371948
664   999c4ffe9ccf530e62c60fac1271a3787b2c20a9  done in  6.569968938827515
665   9a1de1dab15d6ebbac429af9682fcc160beacc74  done in  6.906297922134399
666   9a2c20b7b8602d1dd7332f8fb5017e5c846185c4  done in  8.045441150665283
667   9a48dd7c54b251b4eab96488124e6964ed16c1c7  done in  64.98375797271729
668   9a5fa5e9c10dc40e20610795bad44e7e00391c1f  done in  7.10499906539917
669   9a7e12ef462e779263ee3f2dd415d2d21233f15d  done in  6.911686897277832
670   9af1f7b6d79e6ce314d1842a8d32402328fcaca7  done in  6.764016151428223
671   9bdc4c5f6ba780f30ae081be7af34262af9a60a5  done in  6.857930898666382
672   9c3666efd35f7a8365db74716b0265110c610811  done in  7.046044826507568
673   9c4d7805f3724fb7b866950eaa0505952ff25fd0  done in  36.01916193962097
674   9c7032664c6902c5a936

771   b8c7a947bda15a4f8a4b2f5e8cb8cd35a193fe4c  done in  7.874268293380737
772   b8eea99bb5345329ea058072133f568f7bdd27bd  done in  6.693828105926514
773   b9220fdc68eaa6dce0c5944a769e3e531839055d  done in  37.918243169784546
774   b9413a51b6865a0a2c21993e87428afa76aba440  done in  28.51959204673767
775   b9d502d98e3bb19f8179a4363e3f28c6b6def7a8  done in  6.713598966598511
776   ba69c46c62b525941e91d0788d20b6ef91847cc4  done in  6.753974914550781
777   ba8e624dac40100f10162eaa3d4034904bc08f48  done in  6.599040985107422
778   bb93fdae023ee2978e82f47c1c069b973046ef1e  done in  7.398287057876587
779   bbe8c5e53ca6e4db115afeaaad2be268f039f10d  done in  6.957826852798462
780   bcdd5670761de0087d2d2bb0388da697b0d4348c  done in  6.935347080230713
781   bd034b87414c58fc731c193fdd37f59b3db883fa  done in  6.7830188274383545
782   bd185e4f80d1fa2def96815269ec60dff862a7a1  done in  6.699683904647827
783   bdbefe44ceb31ed05d09c5d322fbdb149abf48e0  done in  6.834239959716797
784   bdd6b6635cf5861fc

881   d6f71b84f703152a08226d1c7e61cab888380fb1  done in  6.498304843902588
882   d76ceda01173508da98f2989844e75fd40859c27  done in  7.065133810043335
883   d7c0189c35a3ee56b13b0488f925df1152a8cfd4  done in  6.395838975906372
884   d7e3a93de209c0c5501b4932b9a7d81c89ed64e8  done in  6.778047800064087
885   d8643f4ada0b138bcbe210f84735eac87b220a07  done in  6.772573232650757
886   d887526eaaaf42627b8dda69ef0c1ae561453b8d  done in  6.857416152954102
887   d8ae4f66a0406ab7167f9ffd39ead29e62bac28f  done in  6.587656021118164
888   d8dea89f5c7fb15d5ba16d2edcd2933a25789ee6  done in  7.2435290813446045
889   d97c2e23c5c443f5040df0943847bab2db491147  done in  6.458456993103027
890   d99bc2544e4b1f33da5bb8010be8485b41a154f9  done in  6.798804998397827
891   da6629447f61ef8b07ce0698b490e3fe7210c1e3  done in  31.968071937561035
892   da803152928a3b98ef21a81a6675e7b51852510e  done in  6.752974271774292
893   da8fcbcb25d73558752e1d2b1f34c0e5df07ab65  done in  6.877561807632446
894   daa9e3b88e01db544

991   fdd141a4fffc490fb3ab570c28b8a1f2dd80a15f  done in  6.749977111816406
992   fdf29d5dec6f929409e0bb340ae973a91680ad17  done in  7.232593059539795
993   fe2533594e01b374ee12f8d450069b21b572a675  done in  7.009338140487671
994   fe4cfc7df897c632311ed0165e66ff9932865e5f  done in  8.981276988983154
995   fe5cc6e84dc223580c0e1729614cfaa8320c5728  done in  6.579675912857056
996   fe63023bb7434e44b4e63dcf918c6a2f9aaba930  done in  6.630901336669922
997   fe76c167920898330a66e3f2509ff1cd41e231bc  done in  6.770268201828003
998   feef5abe52b21ffc198a9f439ec49151c545ef12  done in  7.592700958251953
999   ffaa313b8da3695627cd9915ca46b8bed24a9f4a  done in  7.7234649658203125


In [125]:
len(specter_embeddings)

1000

In [128]:
import json 

temp = json.dumps(specter_embeddings)

# open file for writing, "w" 
f = open("specter_embeddings_updated.json","w")

# write json object to file
f.write(temp)

# close file
f.close()

In [129]:
# Download specter embeddings for query papers from semantic scholar
specter_embeddings_query = {}
for query_id, candidates in all_candidates.items():
        url = 'https://api.semanticscholar.org/graph/v1/paper/{}?fields=embedding'.format(query_id)
        response = requests.get(url, headers=headers)
        specter_embeddings_query[query_id] = response.text

In [165]:
query_embeddings_dict = {}
for k, v in specter_embeddings_query.items():
    temp_dict = json.loads(v)
    if 'embedding' not in temp_dict:
        print(k)
    else:
        query_embeddings_dict[k] = json.loads(v)['embedding']['vector']

0bf046038a555bc848030a28530f9836e5611b96
64334ac9dfb59d68380784e3b1ad197511850921
e414ba960ee2a385b6800f2086209c711cc3b48b


In [168]:
specter_embeddings_dict = {}
count = 0
for k, v in specter_embeddings.items():
    temp_s = {}
    doc_id_recorder = all_candidates[k]
    for i in range(len(v)):
        temp_dict = json.loads(v[i])
        if 'embedding' not in temp_dict:
            print(v[i])
            count += 1
        else:
            temp_s [doc_id_recorder[i]] = json.loads(v[i])['embedding']['vector']
    specter_embeddings_dict[k] = temp_s  
print(count)

{"error":"Paper with id 0541d5338adc48276b3b8cd3a141d799e2d40150 not found"}

{"error":"Paper with id 0541d5338adc48276b3b8cd3a141d799e2d40150 not found"}

{"error":"Paper with id 0541d5338adc48276b3b8cd3a141d799e2d40150 not found"}

{"error":"Paper with id 20739c96ed44ccfdc5352ea38e1a2a15137363f4 not found"}

{"error":"Paper with id 09d1a6f5a50a8c3e066fb05a8833bc00663ada0e not found"}

{"error":"Paper with id 278ab6b7b30db9d013c49bf396404f37dc65c07b not found"}

{"error":"Paper with id 3a9c17cc59386433e743d537abcb7b8076827061 not found"}

{"error":"Paper with id 16fc1065c296840cb0f8ca62601aa17b7f0a02bf not found"}

{"error":"Paper with id 0541d5338adc48276b3b8cd3a141d799e2d40150 not found"}

{"error":"Paper with id 111b0d103542ad8679767e67bbeff8b6504002bc not found"}

{"error":"Paper with id cb745fd78fc7613f95bf5bed1fb125d2e7e39708 not found"}

{"error":"Paper with id 363cfbd6f1e76b1e3431f09e6a1fa1541a4e2162 not found"}

{"error":"Paper with id 7848c7c6a782c6941bdd67556521aa97569366a4

{"message": "Endpoint request timed out"}
{"message": "Endpoint request timed out"}
{"error":"Paper with id 9406ee01e3fda0932168f31cd3835a7d7a943fc6 not found"}

{"error":"Paper with id 281eeacc1d2fdd9a79f652b77afb14d8a3c2d7b6 not found"}

{"error":"Paper with id cad77f0b65e685963de3604fd122105308044aff not found"}

{"error":"Paper with id b2e7c5ddb686d503091dce57efe67fa89480fba4 not found"}

{"error":"Paper with id 281eeacc1d2fdd9a79f652b77afb14d8a3c2d7b6 not found"}

{"message": "Network error communicating with endpoint"}
{"error":"Paper with id ca9a678185f02807ae1b7d9a0bf89247a130d949 not found"}

{"error":"Paper with id c8859b7ac5f466675c41561a6a299f7078a90df0 not found"}

{"message": "Endpoint request timed out"}
{"message": "Network error communicating with endpoint"}
{"error":"Paper with id c11de783900e118e3d3de74efca5435c98b11e7c not found"}

{"message": "Network error communicating with endpoint"}
{"error":"Paper with id 0fa1911622a6c0a3dd43fefbdf2695ebdb7e10fa not found"}

{

In [132]:
temp_query = json.dumps(specter_embeddings_query)

# open file for writing, "w" 
f = open("specter_embeddings_query_updated.json","w")

# write json object to file
f.write(temp_query)

# close file
f.close()

In [155]:
test_queries_id = citation_prediction['query_id'].unique().tolist()

In [156]:
# queries: key: query_id, value: embeddings
# candidates: key: query_id, value: dict(paper_id: embeddins)
def rank_SPECTER(queries, candidates, test_queries_id):
    rankings = {} # store the rankings for each query
    for qid in test_queries_id:
        if qid in queries:
            qemb = queries[qid]
        else:
            continue
        rankings[qid] = {}
        for pid, pemb in candidates[qid].items():
            qemb = np.array(qemb)
            pemb = np.array(pemb)
            dist = np.linalg.norm(qemb - pemb) # calculate L2 distance
            rankings[qid][pid] = dist
        # sort the papers by distance (in ascending order) and return the sorted list
        rankings[qid] = sorted(rankings[qid].items(), key=lambda x: x[1])
    result = []
    for k, v in rankings.items():
        temp = []
        for doc_id, score in v:
            temp.append(doc_id)
        result.append(temp)
    return result

In [214]:
def rank_SPECTER_cosine(queries, candidates, test_queries_id):
    rankings = {} # store the rankings for each query
    for qid in test_queries_id:
        if qid in queries:
            qemb = queries[qid]
        else:
            continue
        rankings[qid] = {}
        for pid, pemb in candidates[qid].items():
            qemb = np.array(qemb)
            pemb = np.array(pemb)
            cosine_similarity_value = cosine_similarity([qemb], [pemb])
            rankings[qid][pid] = cosine_similarity_value
        # sort the papers by distance (in ascending order) and return the sorted list
        rankings[qid] = sorted(rankings[qid].items(), key=lambda x: x[1], reverse = True)
    result = []
    for k, v in rankings.items():
        temp = []
        for doc_id, score in v:
            temp.append(doc_id)
        result.append(temp)
    return result

In [82]:
def calculate_map(relevant_docs, retrieval_results):
    """
    Calculates the mean average precision (MAP) given a list of relevant documents
    and a list of retrieval results.
    
    Inputs:
    relevant_docs: A list of lists, where each inner list contains the relevant
                          document IDs for a query.
    retrieval_results: A list of lists, where each inner list contains the
                              document IDs retrieved by a model for a query.
    
    return: The mean average precision (MAP).
    """
    average_precision = 0
    num_queries = len(relevant_docs)
    
    for i in range(num_queries):
        query_rel_docs = relevant_docs[i]
        query_ret_results = retrieval_results[i]
        
        precision_at_k = []
        num_rel_docs = 0
        
        for k in range(len(query_ret_results)):
            if query_ret_results[k] in query_rel_docs:
                num_rel_docs += 1
                precision_at_k.append(num_rel_docs / (k + 1))
        
        if len(precision_at_k) > 0:
            average_precision += sum(precision_at_k) / len(precision_at_k)
    
    mean_average_precision = average_precision / num_queries
    return mean_average_precision

In [215]:
ranking_results = rank_SPECTER_cosine(query_embeddings_dict, specter_embeddings_dict, test_queries_id)

In [170]:
relevant_docs = []
excluded_queries = ['0bf046038a555bc848030a28530f9836e5611b96',
                   '64334ac9dfb59d68380784e3b1ad197511850921',
                   'e414ba960ee2a385b6800f2086209c711cc3b48b']

for k in test_queries_id: 
    if k in excluded_queries:
        continue
    temp = positive_candidates[k]
    relevant_docs.append(temp)

In [216]:
score = calculate_map(relevant_docs, ranking_results)

In [217]:
score

0.8705778109153777

### Rerank with harmonic

In [116]:
def rerank_harmonic_old(result, harmonic_centralities):
    for i in range(len(result)):
        papers = result[i]
        if harmonic_centralities[i]:
            id_score = [('', 0) for _ in range (30)]
            for i in range(len(papers)):
                doc_id = papers[i]
                score = 0
#                 print(doc_id)
                if doc_id in harmonic_centralities[i]:
                    score = harmonic_centralities[i][doc_id]
                id_score[i] = (doc_id, score)
            id_score = sorted(id_score, key=lambda x: x[1])
            result[i] = [id_score[i][0] for i in range(5)] 
        else:
            result[i] = papers[:5]
    return result

In [97]:
def calculate_harmonic_centralities_old(all_candidates, candidates_edges_dict):
    result = []
    for query_id, candidates in all_candidates.items():
        nodes = candidates
        edges = candidates_edges_dict[query_id]
        
        # Create a graph from the list of edges
        G = nx.Graph(edges)
    
        # Calculate harmonic centralities for each node in the input node list
        harmonic_centralities = nx.harmonic_centrality(G, nodes)
        result.append(harmonic_centralities)
        
    return result

In [117]:
new_result = rerank_harmonic(result, harmonic_centralities)

In [120]:
calculate_map(relevant_docs, new_result)

0.28416666666666673

In [175]:
file_name = 'harmonic_centralities_no_exclusion_total.json'
with open(file_name) as f: 
    harmonic_centralities_no_exclusion = json.load(f)

In [178]:
transformed_harmonic_centralities = []
for score_dict in harmonic_centralities_no_exclusion:
    temp_harmonic = {}
    for key, value in score_dict.items():
        temp_harmonic[key] = math.log(value + 3 * 10 ** -2)
    transformed_harmonic_centralities.append(temp_harmonic)

In [197]:
transformed_harmonic_centralities_shortened = []
for i in range(1000):
    query_id = test_queries_id[i]
    if query_id not in query_embeddings_dict:
        continue
    transformed_harmonic_centralities_shortened.append(transformed_harmonic_centralities[i])

In [220]:
# queries: key: query_id, value: embeddings
# candidates: key: query_id, value: dict(paper_id: embeddins)
def search_SPECTER_rerank(query_emb, candidates_dict):
    result = {}
    for pid, pemb in candidates_dict.items():
        query_emb = np.array(query_emb)
        pemb = np.array(pemb)
        cosine_similarity_value = cosine_similarity([query_emb], [pemb])
        result[pid] = cosine_similarity_value
    return result

In [208]:
def rerank_harmonic(SPECTER_retrieval_results, harmonic_centralities):
    results = []
    for alpha in range(1, 10):
        alpha /= 10
        epoch_result = []
        for i in range(len(SPECTER_retrieval_results)):
            doc_id_score_dict = SPECTER_retrieval_results[i]
            id_score = [('', 0) for _ in range (len(doc_id_score_dict))]
            for j, (doc_id, SPECTER_score) in enumerate(doc_id_score_dict.items()):
                if doc_id in harmonic_centralities[i]:
                    total_score = alpha*harmonic_centralities[i][doc_id]+(1-alpha)*SPECTER_score
                else: 
                    total_score = (1-alpha) * SPECTER_score
                id_score[j] = (doc_id, total_score)
            id_score = sorted(id_score, key=lambda x: x[1], reverse=True)
            epoch_result.append([id_score[i][0] for i in range(5)]) 
        results.append(epoch_result)
    return results

In [205]:
def rerank_final_result(test_queries_id, query_embeddings_dict, positive_candidates, all_candidates, transformed_harmonic_centralities):
    retrieval_results = []
    relevant_docs = []
    scores = []
    for query_id in test_queries_id:
        if query_id in query_embeddings_dict:
            retrieved = search_SPECTER_rerank(query_embeddings_dict[query_id], all_candidates[query_id])
            retrieval_results.append(retrieved)
            relevant_docs.append(positive_candidates[query_id])
    print(len(retrieval_results))
    retrieval_results = rerank_harmonic(retrieval_results, transformed_harmonic_centralities)
    for retrieval_result in retrieval_results:
        score = calculate_map(relevant_docs, retrieval_result)
        scores.append(score)
    return scores

In [221]:
start = time.time()
rerank_result = rerank_final_result(test_queries_id, query_embeddings_dict, positive_candidates, specter_embeddings_dict, transformed_harmonic_centralities_shortened)
end = time.time()
print("Total time used is ", end - start)
print("Re-ranking model MAP score is: ", rerank_result)

997
Total time used is  7.628233909606934
Re-ranking model MAP score is:  [0.98104034325198, 0.9680931683940727, 0.9427309706898496, 0.9229870166053741, 0.9035773988632594, 0.890655299231029, 0.88037167056726, 0.8711119469519694, 0.8638345592332576]


### Rerank with indegree

In [222]:
with open('indegree_dict.json') as f:
    indegree_dict = json.load(f)

In [229]:
transformed_indegree_dict = {}
for k, v in indegree_dict.items():
    transformed_indegree_dict[k] = {}
    for key, value in v.items():
        transformed_indegree_dict[k][key] = math.log(value + 3 * 10 ** -2)

In [225]:
def rerank_indegree(SPECTER_retrieval_results, indegree_dict):
    results = []
    for alpha in range(1, 10):
        alpha /= 10
        epoch_result = []
        for i in range(len(SPECTER_retrieval_results)):
            query_id = test_queries_id[i]
            doc_id_score_dict = SPECTER_retrieval_results[i]
            id_score = [('', 0) for _ in range (len(doc_id_score_dict))]
            for j, (doc_id, SPECTER_score) in enumerate(doc_id_score_dict.items()):
                if doc_id in indegree_dict[query_id]:
                    total_score = alpha*indegree_dict[query_id][doc_id]+(1-alpha)*SPECTER_score
                else: 
                    total_score = (1-alpha) * SPECTER_score
                id_score[j] = (doc_id, total_score)
            id_score = sorted(id_score, key=lambda x: x[1], reverse=True)
            epoch_result.append([id_score[i][0] for i in range(5)]) 
        results.append(epoch_result)
    return results

In [233]:
def rerank_final_result_indegree(test_queries_id, query_embeddings_dict, positive_candidates, all_candidates, transformed_indegree_dict):
    retrieval_results = []
    relevant_docs = []
    scores = []
    for query_id in test_queries_id:
        if query_id in query_embeddings_dict:
            retrieved = search_SPECTER_rerank(query_embeddings_dict[query_id], all_candidates[query_id])
            retrieval_results.append(retrieved)
            relevant_docs.append(positive_candidates[query_id])
    retrieval_results = rerank_indegree(retrieval_results, transformed_indegree_dict)
    for retrieval_result in retrieval_results:
        score = calculate_map(relevant_docs, retrieval_result)
        scores.append(score)
    return scores

In [234]:
start = time.time()
rerank_result = rerank_final_result_indegree(test_queries_id, query_embeddings_dict, positive_candidates, specter_embeddings_dict, transformed_indegree_dict)
end = time.time()
print("Total time used is ", end - start)
print("Re-ranking model MAP score is: ", rerank_result)

Total time used is  8.298100233078003
Re-ranking model MAP score is:  [0.941702886437092, 0.9369720271926922, 0.9338863813663241, 0.93152791708459, 0.9302184330770118, 0.9293811991530175, 0.9286944165830856, 0.9284548088710606, 0.928045246851669]
